# Exploratory Data Analysis

### COVID-19 Daily Infections and Deaths Data
#### Importing dataset

In [43]:
root_path = '/home/mbai/python/covid_ds/'
import sys
sys.path.append(root_path)

import pandas as pd
import numpy as np
import sqlite3
import seaborn as sns

from etl.constants import ETLConfigs
pd.options.mode.chained_assignment = None

conn = sqlite3.connect("{}{}.db".format(root_path, ETLConfigs.DB_NAME))
query = "SELECT * FROM {}".format(ETLConfigs.TABLE_NAME)

df = pd.read_sql_query(query, conn)

### Initial view of the dataset
Let's get a quick view of the scope and type of dataset we're dealing with:  
Looking at the top/bottom/random 20 rows, data shape, and datatypes

In [44]:
df.head()

,country,state,latitude,longitude,date,confirmed,death,etl_load_time
0,Afghanistan,None,33.0000,65.0000,2020-01-22 00:00:00,0,0,2020-04-28 19:58:39.555707
1,Albania,None,41.1533,20.1683,2020-01-22 00:00:00,0,0,2020-04-28 19:58:39.555707
2,Algeria,None,28.0339,1.6596,2020-01-22 00:00:00,0,0,2020-04-28 19:58:39.555707
3,Andorra,None,42.5063,1.5218,2020-01-22 00:00:00,0,0,2020-04-28 19:58:39.555707
4,Angola,None,-11.2027,17.8739,2020-01-22 00:00:00,0,0,2020-04-28 19:58:39.555707


In [45]:
df.tail()

,country,state,latitude,longitude,date,confirmed,death,etl_load_time
341920,US,Utah,38.996171,-110.701396,2020-04-27 00:00:00,1,0,2020-04-28 19:58:39.555707
341921,US,Utah,37.854472,-111.441876,2020-04-27 00:00:00,2,0,2020-04-28 19:58:39.555707
341922,US,Utah,40.124915,-109.517442,2020-04-27 00:00:00,1,0,2020-04-28 19:58:39.555707
341923,US,Utah,41.271160,-111.914512,2020-04-27 00:00:00,3,0,2020-04-28 19:58:39.555707
341924,US,Utah,NaN,NaN,2020-04-27 00:00:00,0,0,2020-04-28 19:58:39.555707


In [46]:
df.sample(20)

,country,state,latitude,longitude,date,confirmed,death,etl_load_time
64908,US,Colorado,37.319409,-102.560322,2020-02-03 00:00:00,0,0,2020-04-28 19:58:39.555707
194678,US,Utah,40.124915,-109.517442,2020-03-13 00:00:00,0,0,2020-04-28 19:58:39.555707
185548,US,California,38.779660,-120.523317,2020-03-11 00:00:00,0,0,2020-04-28 19:58:39.555707
241833,US,Kentucky,37.692414,-83.959564,2020-03-28 00:00:00,0,0,2020-04-28 19:58:39.555707
216538,US,New Mexico,36.481264,-103.470962,2020-03-20 00:00:00,0,0,2020-04-28 19:58:39.555707
288112,US,Montana,46.855389,-112.935916,2020-04-11 00:00:00,0,0,2020-04-28 19:58:39.555707
339046,US,Florida,28.715858,-81.240603,2020-04-27 00:00:00,10,0,2020-04-28 19:58:39.555707
6185,Israel,None,31.000000,35.000000,2020-02-14 00:00:00,0,0,2020-04-28 19:58:39.555707
224156,US,Virginia,37.264206,-77.396097,2020-03-22 00:00:00,0,0,2020-04-28 19:58:39.555707
144853,US,North Carolina,34.893294,-76.541201,2020-02-27 00:00:00,0,0,2020-04-28 19:58:39.555707


In [47]:
df.shape

(341925, 8)

In [48]:
df.dtypes

country           object
state             object
latitude         float64
longitude        float64
date              object
confirmed          int64
death              int64
etl_load_time     object
dtype: object

### A few observations here:

- We have 341k rows of data and 8 columns
- country, state, longitude, and latitude denoting location in the world (labels)
- state has `None` as null value
- confirmed and death for potential prediction (target)
- date and etl_load_time seems to be stored as `string` rather than `datetime`
- `etl_load_time` seems obsolete for this purpose

Let's drop `etl_load_time` and convert `date` to the proper datatype

In [55]:
df['date'] = df['date'].astype('datetime64', copy=False)
df.drop(columns=['etl_load_time'], inplace=True, errors='ignore')
df.replace({"None": np.NaN}, inplace=True)

df.dtypes

country              object
state                object
latitude            float64
longitude           float64
date         datetime64[ns]
confirmed             int64
death                 int64
dtype: object

In [56]:
df.head()

,country,state,latitude,longitude,date,confirmed,death
0,Afghanistan,None,33.0000,65.0000,2020-01-22,0,0
1,Albania,None,41.1533,20.1683,2020-01-22,0,0
2,Algeria,None,28.0339,1.6596,2020-01-22,0,0
3,Andorra,None,42.5063,1.5218,2020-01-22,0,0
4,Angola,None,-11.2027,17.8739,2020-01-22,0,0


### Searching and dealing with *null*

